In [1]:
import pennylane as qml
import warnings

In [2]:
## Include other required notebooks.
%run prepareVQE.ipynb

In [3]:
## Define the net charge of BeH2.
BeHHcharge = 0

In [4]:
h_quiqbox, (n_qubits, n_electrons) = genBeH2(genSTO3Gopt, name="BeH2_quiqbox", grouping_type=None)

In [5]:
## Tapering hamiltonian, Hartree-Fock state, and single/double excitation operators.
## This will take 5 - 10 minutes to run.
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    generators = qml.symmetry_generators(h_quiqbox)
paulixops = qml.paulix_ops(generators, n_qubits)
paulix_sector = qml.qchem.optimal_sector(h_quiqbox, generators, n_electrons)
h_tapered = qml.taper(h_quiqbox, generators, paulixops, paulix_sector)

state_tapered = qml.qchem.taper_hf(generators, paulixops, paulix_sector,
                                   num_electrons=n_electrons,
                                   num_wires=len(h_quiqbox.wires))

singles, doubles = qml.qchem.excitations(n_electrons, len(h_quiqbox.wires))
doubles_tapered = [
    qml.taper_operation(qml.DoubleExcitation, generators, paulixops, paulix_sector,
                        wire_order=h_quiqbox.wires, op_wires=double) for double in doubles
]
singles_tapered = [
    qml.taper_operation(qml.SingleExcitation, generators, paulixops, paulix_sector,
                        wire_order=h_quiqbox.wires, op_wires=single) for single in singles
]

In [6]:
print("\n<Info of Adapt-VQE with qubit tapering>")
print('Number of qubits needed original:', len(h_quiqbox.wires))
print('Number of qubits needed after tapering:', len(h_tapered.wires))


<Info of Adapt-VQE with qubit tapering>
Number of qubits needed original: 14
Number of qubits needed after tapering: 9


In [7]:
print('Number of Pauli strings before tapering:', len(h_quiqbox.ops))
print('Number of Pauli strings after tapering:', len(h_tapered.ops))

Number of Pauli strings before tapering: 666
Number of Pauli strings after tapering: 596


In [8]:
def adapt_circuit_1_tapered(params, excitations):
    qml.BasisState(state_tapered, wires=h_tapered.wires)
    for idx, tapered_op in enumerate(excitations):
        tapered_op(params[idx])
    return qml.expval(h_tapered)

In [9]:
## Circuit for fixing parameters of double excition gates and optimizing parameters of single excitation gates.
def adapt_circuit_2_tapered(params, excitations, gates_select, params_select):
    qml.BasisState(state_tapered, wires=h_tapered.wires)
    
    for idx, tapered_op in enumerate(gates_select):
        tapered_op(params_select[idx])
        
    for idx, tapered_op in enumerate(excitations):
        tapered_op(params[idx])
        
    return qml.expval(h_tapered)

In [10]:
## Construct Adapt-VQE.
dev = qml.device("default.qubit", wires=h_tapered.wires)
diff_method = 'backprop'
cost_fn_final_tapered, params_final_tapered, gates_select_tapered = constructAdaptVQE(dev, 
    diff_method, adapt_circuit_1_tapered, adapt_circuit_2_tapered, doubles_tapered, singles_tapered
)

Original Number of double excitation gates: 180
Number of double excitation gates after filtering: 32
Original Number of single excitation gates: 24
Number of single excitation gates after filtering: 6


In [11]:
## Run the constructed VQE.
## For excitations we'll use our singles and doubles selected gates, i.e., `gates_select`.
maxStep = 50
adamStep = 0.02
opt_adam = qml.AdamOptimizer(stepsize=adamStep, beta1=0.9, beta2=0.99, eps=1e-08)
adapt_energies_tapered, adapt_runtime_tapered = run_adapt_vqe(
    cost_fn_final_tapered, params_final_tapered, opt_adam, maxStep, excitations=gates_select_tapered
)

Completed iteration 5
Energy: -15.614934901761336 Ha
Step Time: 1.1554474830627441 s
----------------
Completed iteration 10
Energy: -15.615748229304543 Ha
Step Time: 1.2724957466125488 s
----------------
Completed iteration 15
Energy: -15.618786788292779 Ha
Step Time: 1.2048327922821045 s
----------------
Completed iteration 20
Energy: -15.620347361028236 Ha
Step Time: 1.0847699642181396 s
----------------
Completed iteration 25
Energy: -15.620786112934882 Ha
Step Time: 1.2269716262817383 s
----------------
Completed iteration 30
Energy: -15.621086038207755 Ha
Step Time: 1.2588098049163818 s
----------------
Completed iteration 35
Energy: -15.621314048607097 Ha
Step Time: 1.0631639957427979 s
----------------
Completed iteration 40
Energy: -15.621339153620434 Ha
Step Time: 1.1233205795288086 s
----------------
Completed iteration 45
Energy: -15.621439984435717 Ha
Step Time: 1.157865285873413 s
----------------
Completed iteration 50
Energy: -15.621475490028745 Ha
Step Time: 1.22570848